# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686542


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:48,  3.89s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:17,  2.86s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:50,  2.03s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:36,  1.54s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:16<00:01,  2.65it/s]

Rendering models:  93%|█████████▎| 28/30 [00:16<00:00,  2.31it/s]

Rendering models:  97%|█████████▋| 29/30 [00:17<00:00,  2.43it/s]

Rendering models: 100%|██████████| 30/30 [00:17<00:00,  2.63it/s]

Blairicus                             0.001084
not-logged-in-30f9608d94e87b00f04a    0.038970
not-logged-in-f7e2260ed4a9517a3e4a    0.000859
Dynamatt                              0.000358
Petrusperes                           0.001497
Camelron                              0.000305
not-logged-in-7e36d1f05e1fdd4c2b64    0.000477
DrPZ                                  0.000329
optimaxdfi                            0.001959
Jessica_Korkmaz                       0.003259
not-logged-in-eff3ba8e9f985ee0895b    0.018894
not-logged-in-214814e52004b42491a4    0.002574
not-logged-in-9f30f411df9d28d2c541    0.002165
acapirala                             0.000703
not-logged-in-f47183e1b336e5121b01    0.001666
not-logged-in-668756376bb84c0481bc    0.003294
Lavadude                              0.023622
TheBlueShadow24                       0.000451
jordanrushworth                       0.002272
not-logged-in-8ba8ad8bc2c1bb4eccca    0.600032
Lewyke                                0.002655
equidad1     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())